In [ ]:
# https://mne.tools/stable/auto_tutorials/inverse/35_dipole_orientations.html

import mne
import numpy as np
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse

data_path = sample.data_path()
meg_path = data_path / "MEG" / "sample"
# evokeds = mne.read_evokeds(meg_path / "sample_audvis-ave.fif")
# evokeds = mne.read_evokeds('/home/zhibinz2/Documents/GitHub/CAMCAN_MEG_100/CC120319/rest/rest_raw.fif')
evokeds = mne.read_evokeds('./test_scripts/evoked-ave.fif')
left_auditory = evokeds[0].apply_baseline()
fwd = mne.read_forward_solution('surface_simple_CAM_fsaverage_fixed_fwd.fif')
mne.convert_forward_solution(fwd, surf_ori=True, copy=False)
# noise_cov = mne.read_cov(meg_path / "sample_audvis-cov.fif")
# noise_cov = mne.read_cov('/home/zhibinz2/Documents/GitHub/CAMCAN_MEG_100/CC120319/rest/rest_raw.fif')
noise_cov = mne.read_cov('./test_scripts/CAMCAN-cov.fif')
subject = "fsaverage"
subjects_dir = data_path / "subjects"
# trans_fname = 'sample_fsaverage_manual_trans.fif'
trans_fname = './test_scripts/CAMCAN_fsaverage_trans.fif'
